Задание 1.

Обучите нейронную сеть решать шифр Цезаря.

Что необходимо сделать:

* Написать алгоритм шифра Цезаря для генерации выборки (сдвиг на К каждой буквы. Например, при сдвиге на 2 буква “А” переходит в букву “В” и тп)
* Сделать нейронную сеть
* Обучить ее (вход - зашифрованная фраза, выход - дешифрованная фраза)
* Проверить качество

Задание 2.

Выполнить практическую работу из лекционного ноутбука.

* Построить RNN-ячейку на основе полносвязных слоев
* Применить построенную ячейку для генерации текста с выражениями героев сериала “Симпсоны”

In [1]:
import pandas as pd  # для работы с данными
import time  # для оценки времени
import torch  # для написания нейросети
import numpy as np
import re

In [2]:
def f_cifer_cesar(phrase,num_K):

    abvgd = 'abcdefghijklmnopqrstuvwxyz'
    if num_K>len(abvgd):
        num_K=len(abvgd)

    ab_ba=abvgd+abvgd[::-1]

    w_i_dict = {w: i for i, w in enumerate(abvgd)}

    loc_lst=''
    for letter in phrase:
        if letter in abvgd:
            loc_lst+=ab_ba[w_i_dict[letter]+num_K]
        else:
            loc_lst+=letter

    return loc_lst

In [3]:
def f_clean_text(phrase):

    abvgd = 'abcdefghijklmnopqrstuvwxyz '

    w_i_dict = {w: i for i, w in enumerate(abvgd)}

    loc_lst=''
    for letter in phrase.lower():
        if letter in abvgd:
            loc_lst+=abvgd[w_i_dict[letter]]
        else:
            loc_lst+=''

    return loc_lst

In [4]:
def f_text_to_num(phrase):

    abvgd = 'abcdefghijklmnopqrstuvwxyz '

    w_i_dict = {w: i for i, w in enumerate(abvgd)}

    loc_lst=[]
    for letter in phrase.lower():
        if letter in abvgd:
            loc_lst.append(w_i_dict[letter])

    return torch.Tensor(loc_lst)

In [81]:
def f_num_to_text(lst):

    abvgd = 'abcdefghijklmnopqrstuvwxyz '
    lst=list(lst.detach().numpy())
    i_w_dict = {i: w for i, w in enumerate(abvgd)}

    loc_lst=''
    for num in lst:
        if num in range(len(abvgd)):
            loc_lst+=i_w_dict[num]

    return loc_lst

In [6]:
def f_cadr_text(phrase,lenght):

    loc_lst=phrase
    for i in range(lenght-len(phrase)):
        loc_lst+=' '

    return loc_lst

In [7]:
df = pd.read_csv('data.csv',usecols=['normalized_text'])

df=df.dropna().reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10891 entries, 0 to 10890
Data columns (total 1 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   normalized_text  10891 non-null  object
dtypes: object(1)
memory usage: 85.2+ KB


In [8]:
df['clean_text']=[f_clean_text(x) for x in df['normalized_text']]


In [9]:
lst_max=[len(x) for x in df['clean_text']]

lenght_max=lst_max[np.argmax(lst_max)]

lenght_max,'\n',df['clean_text'][np.argmax(lst_max)]

(392,
 '\n',
 'elegy for geezer rock postcard image thing to see to think of springfield is to think of thee what thoughts bepass ahind thy mien why sky art blue why trees art green and what pray tell did thine eyes see perchance old friend they gazed at me brought low by natures oafish hand thou crushed our reviewing stand and twixt thy stones glimpsed i the truth all things must pass  thy face my youth')

In [10]:
len(f_cadr_text(df['clean_text'][0],lenght_max))

392

In [11]:
df['cadr_text']=[f_cadr_text(x,lenght_max) for x in df['clean_text']]

In [12]:
K_cifer=25

In [13]:
df['cifer_text']=[f_cifer_cesar(x,K_cifer) for x in df['cadr_text']]
df.head()

,normalized_text,clean_text,cadr_text,cifer_text
0,maggie look whats that,maggie look whats that,maggie look whats that ...,ozuusw pmmq etzhi htzh ...
1,lee-mur lee-mur,leemur leemur,leemur leemur ...,pwwogj pwwogj ...
2,zee-boo zee-boo,zeeboo zeeboo,zeeboo zeeboo ...,bwwzmm bwwzmm ...
3,im trying to teach maggie that nature doesnt e...,im trying to teach maggie that nature doesnt e...,im trying to teach maggie that nature doesnt e...,so hjcsnu hm hwzyt ozuusw htzh nzhgjw xmwinh w...
4,its like an ox only it has a hump and a dewlap...,its like an ox only it has a hump and a dewlap...,its like an ox only it has a hump and a dewlap...,shi psqw zn md mnpc sh tzi z tgol znx z xwepzl...


In [14]:
df=df[['cadr_text','cifer_text']]
df.head()

,cadr_text,cifer_text
0,maggie look whats that ...,ozuusw pmmq etzhi htzh ...
1,leemur leemur ...,pwwogj pwwogj ...
2,zeeboo zeeboo ...,bwwzmm bwwzmm ...
3,im trying to teach maggie that nature doesnt e...,so hjcsnu hm hwzyt ozuusw htzh nzhgjw xmwinh w...
4,its like an ox only it has a hump and a dewlap...,shi psqw zn md mnpc sh tzi z tgol znx z xwepzl...


In [15]:
len(df['cadr_text'][0])

392

In [16]:
df['cifer_vec']=[f_text_to_num(x) for x in df['cifer_text']]
df['cadr_vec']=[f_text_to_num(x) for x in df['cadr_text']]



In [17]:
len(df['cifer_vec'][0])

392

In [18]:
df.head()

,cadr_text,cifer_text,cifer_vec,cadr_vec
0,maggie look whats that ...,ozuusw pmmq etzhi htzh ...,"[tensor(14.), tensor(25.), tensor(20.), tensor...","[tensor(12.), tensor(0.), tensor(6.), tensor(6..."
1,leemur leemur ...,pwwogj pwwogj ...,"[tensor(15.), tensor(22.), tensor(22.), tensor...","[tensor(11.), tensor(4.), tensor(4.), tensor(1..."
2,zeeboo zeeboo ...,bwwzmm bwwzmm ...,"[tensor(1.), tensor(22.), tensor(22.), tensor(...","[tensor(25.), tensor(4.), tensor(4.), tensor(1..."
3,im trying to teach maggie that nature doesnt e...,so hjcsnu hm hwzyt ozuusw htzh nzhgjw xmwinh w...,"[tensor(18.), tensor(14.), tensor(26.), tensor...","[tensor(8.), tensor(12.), tensor(26.), tensor(..."
4,its like an ox only it has a hump and a dewlap...,shi psqw zn md mnpc sh tzi z tgol znx z xwepzl...,"[tensor(18.), tensor(7.), tensor(8.), tensor(2...","[tensor(8.), tensor(19.), tensor(18.), tensor(..."


In [19]:
from sklearn.model_selection import  train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['cifer_vec'], df['cadr_vec'], test_size=0.2, random_state=42)

In [20]:
X_train=X_train.reset_index(drop=True)
X_test=X_test.reset_index(drop=True)
y_train=y_train.reset_index(drop=True)
y_test=y_test.reset_index(drop=True)

In [21]:
X_train=pd.Series([torch.Tensor(x).reshape(1,1,392) for x in X_train])
X_test= pd.Series([torch.Tensor(x).reshape(1,1,392) for x in  X_test])
y_train=pd.Series([torch.Tensor(x).reshape(1,1,392) for x in y_train])
y_test= pd.Series([torch.Tensor(x).reshape(1,1,392) for x in  y_test])

In [22]:
X_test.head()

0    [[[tensor(4.), tensor(22.), tensor(15.), tenso...
1    [[[tensor(13.), tensor(12.), tensor(26.), tens...
2    [[[tensor(7.), tensor(19.), tensor(25.), tenso...
3    [[[tensor(25.), tensor(13.), tensor(2.), tenso...
4    [[[tensor(7.), tensor(19.), tensor(25.), tenso...
dtype: object

In [23]:
len(X_test)

2179

In [24]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    for i in range(0, num_examples, batch_size):
        locX_tens=features[i]
        locY_tens=labels[i]

        if (i+batch_size)>num_examples:
            continue

        j = indices[i+1: i + batch_size]#min(i + batch_size, num_examples)]
        for k in j:
            locX_tens=torch.cat((locX_tens,features[k]))
            locY_tens=torch.cat((locY_tens,features[k]))

        yield locX_tens,locY_tens #features[j], labels[j]

In [25]:
x=torch.Tensor(1,1,392)
x.shape

torch.Size([1, 1, 392])

In [ ]:
batch_size = 256
for X, y in data_iter(batch_size, X_train, y_train):
    print(X.shape,'\n', y.shape)
    #break

In [69]:
'''model = torch.nn.Conv2d(in_channels=1,
                         out_channels=1,
                         kernel_size=(1,1))'''

'''model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(392, 2048),
    torch.nn.Sigmoid(),
    torch.nn.Dropout(0.1),
    torch.nn.Linear(2048, 1024),
    torch.nn.Sigmoid(),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(1024, 512),
    torch.nn.Sigmoid(),
    torch.nn.Dropout(0.3),
    torch.nn.Linear(512, 392)
)'''

model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(392, 1512),

    torch.nn.ReLU(),
    torch.nn.Dropout(0.3),
    torch.nn.Linear(1512, 392))
#torch.nn.Flatten(),torch.nn.BatchNorm1d(512),


In [39]:
#model = model.to(device)

In [61]:
import torch.optim.lr_scheduler as lr_scheduler

In [92]:
'''optimizer = torch.optim.SGD(conv2d.parameters(), lr=0.001)
criterion = torch.nn.MSELoss()'''
loss = torch.nn.MSELoss(reduction='mean')
trainer = torch.optim.Adam(model.parameters(), lr=.1)
scheduler1 = lr_scheduler.CosineAnnealingLR(trainer, T_max=5, eta_min=0.00001)

In [71]:
for X, y in data_iter(1, X_train, y_train):
            trainer.zero_grad()
            y_pred = model(X)
            print(X.shape,y.shape)
            break
            l = loss(y_pred, y)

torch.Size([1, 1, 392]) torch.Size([1, 1, 392])


In [93]:
'''m = torch.nn.Conv1d(16, 33, 3, stride=2)
input = torch.randn(20, 16, 50)
output = m(input)

input.shape'''
num_epochs = 10
batch_size=1

def train_model():

    for ep in range(num_epochs):
        train_iters, train_passed  = 0, 0
        train_loss, train_acc = 0., 0.
        start=time.time()
        model.to(device)
        model.train()
        for X, y in data_iter(batch_size, X_train, y_train):
            X, y = X.to(device), y.to(device)
            trainer.zero_grad()
            y_pred = model(X).squeeze()
            l = loss(y_pred, y.squeeze())
            l.backward()
            trainer.step()
            train_loss += l.item()
            train_acc += (y_pred == y.squeeze()).sum().item()#(y_pred.argmax(dim=1) == y).sum().item()
            train_iters += 1
            train_passed += len(X)
        scheduler1.step()

        test_iters, test_passed  = 0, 0
        test_loss, test_acc = 0., 0.
        model.eval()
        for X, y in data_iter(batch_size, X_test, y_test):
            X, y = X.to(device), y.to(device)
            y_pred = model(X).squeeze()
            l = loss(y_pred, y.squeeze())
            test_loss += l.item()
            test_acc += (y_pred == y.squeeze()).sum().item()#(y_pred.argmax(dim=1) == y).sum().item()
            test_iters += 1
            test_passed += len(X)

        print("epoch: {}, taked_time: {:.3f}, train_loss: {:.3f}, train_acc: {:.3f}, test_loss: {:.3f}, test_acc: {:.3f}".format(
            ep, time.time() - start, train_loss / train_iters, train_acc / train_passed,
            test_loss / test_iters, test_acc / test_passed)
        )

In [94]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [95]:
num_epochs = 100
batch_size=3
train_model()

epoch: 0, taked_time: 5.206, train_loss: 12.885, train_acc: 74.311, test_loss: 12.744, test_acc: 101.840
epoch: 1, taked_time: 5.007, train_loss: 12.879, train_acc: 76.209, test_loss: 12.738, test_acc: 92.849
epoch: 2, taked_time: 5.318, train_loss: 12.862, train_acc: 59.556, test_loss: 12.724, test_acc: 72.879
epoch: 3, taked_time: 4.992, train_loss: 12.841, train_acc: 37.301, test_loss: 12.709, test_acc: 36.933
epoch: 4, taked_time: 5.222, train_loss: 12.824, train_acc: 18.149, test_loss: 12.699, test_acc: 19.972


KeyboardInterrupt: ignored

In [91]:
for X, y in data_iter(1, X_test, y_test):
            X, y = X.to(device), y.to(device)
            y_pred = model(X).squeeze()
            print(f_num_to_text(y_pred.cpu()))
            print(y_pred.cpu())
            print(y.squeeze().cpu())
            print(f_num_to_text(y.squeeze().cpu()))
            break

                                                          
tensor([13.8266, 15.1257, 16.3841, 16.7823, 17.1335, 16.1751, 16.7249, 16.6230,
        17.9413, 17.0111, 16.7309, 18.0502, 17.2512, 17.3555, 17.4240, 17.7921,
        18.1841, 18.2853, 18.5659, 18.5524, 19.2311, 18.7831, 18.8119, 18.9822,
        19.9260, 19.0409, 19.4492, 19.4401, 19.7555, 20.7239, 20.0486, 19.6701,
        20.5144, 20.5081, 20.6966, 20.9019, 21.3674, 21.3624, 20.9439, 21.2727,
        21.5295, 22.0904, 21.7390, 21.7184, 21.8689, 22.3604, 22.4757, 22.0777,
        23.0044, 21.7999, 22.4731, 23.1213, 22.5124, 22.4961, 22.7040, 22.2558,
        23.2398, 22.5105, 22.9352, 23.3436, 23.2367, 22.8986, 23.0091, 23.2864,
        23.4742, 23.2173, 23.8852, 23.6179, 23.3301, 23.5754, 23.8239, 23.8092,
        23.9335, 23.6953, 23.9190, 24.1128, 23.8021, 23.7330, 24.0866, 24.2880,
        24.3891, 24.6866, 24.2134, 24.0933, 24.3654, 24.0659, 24.0428, 24.6825,
        24.2463, 24.5220, 24.8374, 24.3454, 24.7765, 24.5958,

In [ ]:
phrases = df['normalized_text'].tolist()  # колонка с предобработанными текстами
phrases[:3]

['maggie look whats that', 'lee-mur lee-mur', 'zee-boo zee-boo']

In [ ]:
text = [[c for c in ph] for ph in phrases[:3] if type(ph) is str]
text

In [ ]:
CHARS = set('abcdefghijklmnopqrstuvwxyz ')  # все символы, которые мы хотим использовать для кодировки = наш словарь
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]  # все неизвестные символы будут получать тег none
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}  # словарь токен-индекс

In [ ]:
INDEX_TO_CHAR

In [ ]:
MAX_LEN = 50  # мы хотим ограничить максимальную длину ввода
X = torch.zeros((len(text), MAX_LEN), dtype=int)  # создаём пустой вектор для текста, чтобы класть в него индексы токенов
for i in range(len(text)):  # для каждого предложения
    for j, w in enumerate(text[i]):  # для каждого токена
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])